In [ ]:
# The Oracle 
# v3.0.9.2 2024-12-06

import re
import os
import base64
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from utils.common_utils import print_help
from ioc_processing.ioc_functions import (
    #validate_iocs,
    parse_bulk_iocs,
    process_individual_ioc_file,
    analysis,
    calculate_total_malicious_score
)
from file_operations.file_utils import read_file, clean_input, is_ip, is_url, is_domain, is_hash, sanitize_and_defang, is_cve, is_org, is_port, is_product


def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to list files in the input_files directory
def list_input_files():
    input_dir = 'input_files'
    return [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Check for light or dark mode and update text color accordingly
js_code = """
<script>
    function applyColorScheme() {
        const isDarkMode = window.matchMedia('(prefers-color-scheme: dark)').matches;
        const elements = document.querySelectorAll('body, .custom-box, .widget-radio-box, .widget-button, .widget-checkbox label, .widget-textarea');

        elements.forEach(el => {
            if (isDarkMode) {
                el.classList.remove('light-mode');
                el.classList.add('dark-mode');
            } else {
                el.classList.remove('dark-mode');
                el.classList.add('light-mode');
            }
        });
    }

    window.matchMedia('(prefers-color-scheme: dark)').addEventListener('change', applyColorScheme);
    applyColorScheme();  // Initial call to set the colors based on current theme
</script>
"""
display(HTML(js_code))


def classify_ioc(ioc):
    ioc = ioc.strip()
    
    # Check for product with port specified (e.g., "prod:product, port")
    if ioc.lower().startswith('prod:'):
        product_info = ioc[5:].strip()
        parts = product_info.split(',')
        product = parts[0].strip() if parts else None
        port = parts[1].strip() if len(parts) > 1 else None
        return 'product_with_port', (product, port)
    
    # Handle other types
    if ioc.lower().startswith('org:'):
        return 'org', ioc[4:].strip()
    elif is_ip(ioc):
        return 'ip', ioc
    elif is_url(ioc):
        return 'url', ioc
    elif is_domain(ioc):
        return 'domain', ioc
    elif is_hash(ioc):
        return 'hash', ioc
    elif is_cve(ioc):
        return 'cve', ioc
    elif is_port(ioc):
        return 'port', ioc
    elif is_org(ioc):
        return 'org', ioc
    else:
        return 'unknown', ioc

# Function to handle validation and display results
def start_validation(output_to_file, custom_output_filename=None, raw_input=None, file_input=None, 
                     file_name_input=None, progress_bar=None, status_output=None, 
                     cve_text=None, selected_country=None, org_text=None, output_format='Text', port_text=None):
    
    #print(f"DEBUG: In start_validation, selected_country = {selected_country}")
    #print(f"DEBUG: org_text passed into start_validation: {org_text}")
    #print(f"DEBUG: output_format = {output_format}")

    # Define plural mapping to ensure correct plural keys
    plural_mapping = {
        'ip': 'ips',
        'url': 'urls',
        'domain': 'domains',
        'hash': 'hashes',
        'cve': 'cves',
        'org': 'orgs',
        'port': 'ports',
        'product': 'products'
    }

    # Initialize product and port text variables
    product_text = None
    port_text = None

    # Prepare output file path if needed
    output_file_path = None
    if output_to_file:
        script_directory = os.getcwd()
        output_directory = os.path.join(script_directory, 'output_files')
        ensure_directory_exists(output_directory)
        file_extension = 'csv' if output_format == 'CSV' else 'txt'
        if output_format == 'Word':
            file_extension = 'docx'
        elif output_format == 'PDF':
            file_extension = 'pdf'
        elif output_format == 'JSON':
            file_extension = 'json'
        elif output_format == 'HTML':
            file_extension = 'html'
        output_file_name = f"{custom_output_filename}.{file_extension}" if custom_output_filename else f"default_output.{file_extension}"
        output_file_path = os.path.join(output_directory, output_file_name)

    # Set country filter to None if 'All' is selected
    selected_country = None if selected_country == 'All' else selected_country

    # Initialize lists for different types of IOCs
    cve_list = []
    product_port_combinations = []
    iocs = []

    # Parse raw_input and classify each IOC
    if raw_input:
        components = [ioc.strip() for ioc in raw_input.split(',') if ioc.strip()]
        for component in components:
            ioc_type, ioc_value = classify_ioc(component)
            if ioc_type == 'cve':
                cve_list.append(ioc_value)
            elif ioc_type == 'product_with_port':
                product, port = ioc_value
                product_port_combinations.append((product, port))
            else:
                # If not CVE or product_with_port, add to general IOC list
                iocs.append(component)

    # Add CVEs from `cve_text` if provided
    if cve_text:
        cve_list.extend([cve.strip() for cve in cve_text.split(',') if cve.strip()])

    # Process CVE analysis if CVEs are present
    if cve_list:
        print(f"DEBUG: Processing CVEs: {cve_list}")
        analysis_args = {
            'selected_category': {'cves': cve_list},
            'output_file_path': output_file_path,
            'progress_bar': progress_bar,
            'status_output': status_output,
            'selected_country': selected_country
        }
        aggregated_report = analysis(**analysis_args)

        with output:
            clear_output()
            print(aggregated_report)
        if status_output:
            with status_output:
                clear_output()
                display(progress_bar)
                display(HTML('<b>Processing complete.</b>'))

    # Process product-port combinations if present
    if product_port_combinations:
        print(f"DEBUG: Processing Product-Port combinations: {product_port_combinations}")
        analysis_args = {
            'selected_category': {'product_port_combinations': product_port_combinations},
            'output_file_path': output_file_path,
            'progress_bar': progress_bar,
            'status_output': status_output,
            'selected_country': selected_country
        }
        aggregated_report = analysis(**analysis_args)

        with output:
            clear_output()
            print(aggregated_report)
        if status_output:
            with status_output:
                clear_output()
                display(progress_bar)
                display(HTML('<b>Processing complete.</b>'))

    # Handle other IOC types from general IOC list if any
    if iocs:
        # Classify each general IOC and build a dictionary
        ioc_dict = {'ips': [], 'urls': [], 'domains': [], 'hashes': [], 'orgs': [], 'ports': [], 'products': []}
        for ioc in iocs:
            ioc_type, ioc_value = classify_ioc(ioc)
            if ioc_type in plural_mapping:
                ioc_dict[plural_mapping[ioc_type]].append(ioc_value)  # Pluralize type to fit dictionary keys

        # Initialize dictionary for categorized IOCs
        ioc_dict = {v: [] for v in plural_mapping.values()}
    
        # Process raw input IOCs if any
        if raw_input:
            iocs = [ioc.strip() for ioc in raw_input.split(',') if ioc.strip()]
            for ioc in iocs:
                ioc_type, ioc_value = classify_ioc(ioc)
                if ioc_type in plural_mapping:
                    ioc_dict[plural_mapping[ioc_type]].append(ioc_value)  # Use the plural form from mapping


        # Filter out empty lists
        ioc_dict = {k: v for k, v in ioc_dict.items() if v}
        
        if ioc_dict:
            print(f"DEBUG: Processing general IOCs: {ioc_dict}")
            analysis_args = {
                'selected_category': ioc_dict,
                'output_file_path': output_file_path,
                'progress_bar': progress_bar,
                'status_output': status_output,
                'selected_country': selected_country
            }
            aggregated_report = analysis(**analysis_args)

            with output:
                clear_output()
                print(aggregated_report)
            if status_output:
                with status_output:
                    clear_output()
                    display(progress_bar)
                    display(HTML('<b>Processing complete.</b>'))

    # Handle organization input separately if provided
    if org_text:
        print(f"DEBUG: Organization input provided: {org_text}")
        analysis_args = {
            'selected_category': {'orgs': [org_text]},
            'output_file_path': output_file_path,
            'progress_bar': progress_bar,
            'status_output': status_output,
        }
        aggregated_report = analysis(**analysis_args)

        with output:
            clear_output()
            print(aggregated_report)
        if status_output:
            with status_output:
                clear_output()
                display(progress_bar)
                display(HTML('<b>Processing complete.</b>'))

    # Handle file input if provided
    if file_input:
        print(f"DEBUG: Uploaded/Selected file content:\n{file_input}")
        file_iocs = [ioc.strip() for ioc in file_input.splitlines() if ioc.strip()]
        
        # Classify IOCs from file and proceed as above
        file_ioc_dict = {'ips': [], 'urls': [], 'domains': [], 'hashes': [], 'cves': [], 'orgs': [], 'ports': [], 'products': []}
        for ioc in file_iocs:
            ioc_type, ioc_value = classify_ioc(ioc)
            if ioc_type != 'unknown':
                file_ioc_dict[ioc_type + 's'].append(ioc_value)
        
        # Filter out empty lists
        file_ioc_dict = {k: v for k, v in file_ioc_dict.items() if v}

        if file_ioc_dict:
            print(f"DEBUG: Processing IOCs from file: {file_ioc_dict}")
            analysis_args = {
                'selected_category': file_ioc_dict,
                'output_file_path': output_file_path,
                'progress_bar': progress_bar,
                'status_output': status_output,
                'selected_country': selected_country
            }
            aggregated_report = analysis(**analysis_args)

            with output:
                clear_output()
                print(aggregated_report)
            if status_output:
                with status_output:
                    clear_output()
                    display(progress_bar)
                    display(HTML('<b>Processing complete.</b>'))

    # Save output if specified
    if output_to_file and output_file_path:
        if output_format == 'CSV':
            save_to_csv(output_file_path, aggregated_report)
        elif output_format == 'Word':
            save_to_word(output_file_path, aggregated_report)
        elif output_format == 'PDF':
            save_to_pdf(output_file_path, aggregated_report)
        elif output_format == 'JSON':
            save_to_json(output_file_path, aggregated_report)
        elif output_format == 'HTML':
            save_to_html(output_file_path, aggregated_report)
        else:
            write_clean_text_output(output_file_path, aggregated_report)

def save_to_word(file_path, report):
    from docx import Document
    doc = Document()
    doc.add_heading('Analysis Report', level=1)
    lines = report.strip().split('\n')
    for line in lines:
        doc.add_paragraph(line)
    doc.save(file_path)

def save_to_pdf(file_path, report):
    from fpdf import FPDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    lines = report.strip().split('\n')
    for line in lines:
        pdf.cell(0, 10, txt=line, ln=True)
    pdf.output(file_path)

def save_to_json(file_path, report):
    import json

    # Convert the report into a structured format if it's unstructured text
    structured_data = {}
    lines = report.strip().split('\n')
    for line in lines:
        if ':' in line:  # Check for key-value format
            key, value = line.split(':', 1)
            structured_data[key.strip()] = value.strip()
        else:
            # Add lines without colons as standalone entries
            structured_data[line.strip()] = None

    # Write the structured data as JSON
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(structured_data, json_file, indent=4)

def save_to_html(file_path, report):
    html_content = "<html><head><title>Analysis Report</title></head><body>"
    html_content += "<h1>Analysis Report</h1>"
    lines = report.strip().split('\n')
    for line in lines:
        html_content += f"<p>{line}</p>"
    html_content += "</body></html>"

    with open(file_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)


def write_clean_text_output(file_path, report):
    lines = report.strip().split('\n')

    with open(file_path, 'w', encoding='utf-8') as outfile:
        for line in lines:
            # Remove leading dashes, periods (..........), and strip extra whitespace
            cleaned_line = line.replace('........', '').lstrip('- ').strip()

            # Write the cleaned line to the file only if it's not empty
            if cleaned_line:
                outfile.write(f"{cleaned_line}\n")

# Function to save the report to a CSV file
def save_to_csv(file_path, report):
    import csv
    lines = report.strip().split('\n')
    
    # Open the CSV file in write mode
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        for line in lines:
            # Remove lines that are made up of periods (the header)
            if set(line.strip()) == {'.'}:
                continue  # Skip this line if it contains only periods
            
            # Remove the first occurrence of "- " (dash and space) and strip extra spaces
            cleaned_line = line.replace('- ', '', 1).strip()
            
            # Split the line by the first colon only if it exists
            if ':' in cleaned_line:
                key, value = cleaned_line.split(':', 1)
                writer.writerow([key.strip(), value.strip()])
            else:
                # If there's no colon, just write the key alone
                writer.writerow([cleaned_line])

# Function to reset the entire form including file inputs
def full_reset_inputs(b):
    output_to_file_checkbox.value = False
    file_input.value = ()  # Reset file input
    file_name_input.value = None  # Reset dropdown selection
    custom_file_name.value = ''  # Reset custom file name
    raw_input.value = ''
    cve_country_dropdown.value = 'All'  # Reset country dropdown to 'All'
    with output:
        clear_output()
    with status_output:
        clear_output()
    processing_label.value = ""  # Clear the processing message
    print("Full reset completed.")  # Debugging message

full_reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='danger',
    tooltip='Click to fully reset inputs',
    icon='trash',
    layout=widgets.Layout(width='500px', height='auto')
)


# Create widgets with custom styles
style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto')

output_to_file_checkbox = widgets.Checkbox(
    value=False,
    description='Save output to file',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', height='auto', padding='18px', overflow='hidden', background_color='transparent', border_radius='10px')  # Adjust height and overflow
)

custom_file_name = widgets.Text(
    value='',
    placeholder='File Name (no ext)',
    description='Output File Name:',
    style=style,
    layout=widgets.Layout(width='270px', background_color='transparent'),
    disabled=True  # Initially disabled
)


# Add a dropdown to select the output file format (CSV or Text)
output_format_dropdown = widgets.Dropdown(
    options=['Text', 'CSV', 'Word', 'PDF', 'JSON', 'HTML'],
    value='Text',
    description='Output Format:',
    style=style,
    layout=widgets.Layout(width='270px', background_color='transparent'),
    disabled=True  # Initially disabled
)

# Function to enable/disable custom file name and output format based on checkbox
def on_output_to_file_change(change):
    if change['new']:  # If checkbox is checked
        custom_file_name.disabled = False
        output_format_dropdown.disabled = False
    else:  # If checkbox is unchecked
        custom_file_name.disabled = True
        output_format_dropdown.disabled = True


# Adjust the layout of the VBox containing the checkbox
checkbox_vbox = widgets.VBox(
    [output_to_file_checkbox, custom_file_name, output_format_dropdown],  # Added custom file name widget
    layout=widgets.Layout(padding='5px', background_color='#333333', border_radius='10px', overflow='hidden')  # Ensure no scroll bars by setting overflow to hidden
)

# Attach observer to the 'output_to_file_checkbox' to detect changes
output_to_file_checkbox.observe(on_output_to_file_change, names='value')

file_input = widgets.FileUpload(
    accept='.txt',  # Accept only .txt files
    multiple=False,  # Accept single file
    style=style,
    layout=widgets.Layout(width='275px')
)

file_name_input = widgets.Dropdown(
    options=list_input_files(),
    value=None,
    placeholder='From input_files directory',
    description='File Name:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

raw_input = widgets.Text(
    value='',
    placeholder='Enter IOC',
    description='Raw Input:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

submit_button = widgets.Button(
    description='Start Validation',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to start validation',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
    style=style,
    layout=widgets.Layout(width='275px')
)


# Full list of countries with an "All" option for global search
country_options = [
    'All', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 
    'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 
    'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 
    'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo (Congo-Brazzaville)', 
    'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 
    'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 
    'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 
    'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 
    'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 
    'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 
    'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 
    'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar (Burma)', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand', 
    'Nicaragua', 'Niger', 'Nigeria', 'North Korea', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine', 
    'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Rwanda', 
    'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 
    'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 
    'Somalia', 'South Africa', 'South Korea', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 
    'Syria', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 
    'Turkey', 'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 
    'Uzbekistan', 'Vanuatu', 'Vatican City', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe'
]

# Create a dropdown for country selection (for CVE searches) with "All" option
cve_country_dropdown = widgets.Dropdown(
    options=country_options,
    value='All',
    description='Country:',
    tooltip='Only available for CVE/Port searches',
    disabled=True,  # Disable later when necessary
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

cve_search_input = widgets.Text(
    value='',
    placeholder='(e.g., CVE-2021-34527)',
    description='CVE Input:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='275px', background_color='transparent')
)

# Function to enable/disable country dropdown based on CVE detection in raw input
def on_raw_input_change(change):
    raw_text = change['new'].strip()
    print(f"DEBUG: Raw input changed to: {raw_text}")
    
    # Split input by commas, trim whitespace, and classify each component
    components = [ioc.strip() for ioc in raw_text.split(',') if ioc.strip()]
    
    # Check if any component is classified as a 'cve', 'product_with_port', or a valid port
    has_valid_entry = any(
        classify_ioc(component)[0] in ['cve', 'product_with_port', 'port']
        for component in components
    )

    # Enable the country dropdown if any valid CVE, product-port, or port is detected
    cve_country_dropdown.disabled = not has_valid_entry
    if has_valid_entry:
        print("DEBUG: CVE(s), product-port, or port detected, country dropdown enabled.")
    else:
        print("DEBUG: No valid CVE, product-port, or port detected, country dropdown disabled.")

# Attach the observer to `raw_input` to detect changes in the raw input field and enable/disable the country dropdown accordingly
raw_input.observe(on_raw_input_change, names='value')

def on_country_change(change):
    print(f"DEBUG: Country selection changed to: {change['new']}")

cve_country_dropdown.observe(on_country_change, names='value')

# Attach observer for raw_input to detect CVEs
raw_input.observe(on_raw_input_change, names='value')

# Load image
image_path = 'oracle_logo.jpg'  # Replace with your actual image path
with open(image_path, "rb") as file:
    image_data = file.read()
    encoded_image = base64.b64encode(image_data).decode('utf-8')

# Create an image widget
image_widget = widgets.Image(
    value=image_data,
    format='jpg',
    width=400,
    height=400,
    layout=widgets.Layout(margin='160px 0 0 0')
)

# Load the banner image
banner_img_path = "The_Oracle_Banner_IOC_Validation_Tool2.jpg"
with open(banner_img_path, "rb") as file, open(image_path, "rb"):
    banner_image_data = file.read()

# Create an image widget for the banner
banner_widget = widgets.Image(
    value=banner_image_data,
    format='jpeg',
    width=800,
    layout=widgets.Layout(margin='0 0 0px 0')
)

# Initialize the progress bar widget
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', layout=widgets.Layout(width='90%', align_self='flex-start'))


processing_label = widgets.Label(value="", layout=widgets.Layout(width='250px', margin='-5px 0 0 0'))  # Label to indicate processing status

output = widgets.Output(layout=widgets.Layout(width='100%', height='1100px', overflow='auto'))  # Increased height to 900px
status_output = widgets.Output(layout=widgets.Layout(width='100%', height='100px'))  # Adjusted height to fit status and progress bar

# Define the interaction logic
def on_submit_button_clicked(b):
    output_to_file = output_to_file_checkbox.value
    raw_text = raw_input.value.strip()  # Get raw text input and trim whitespace
    cve_text = cve_search_input.value.strip()  # Get CVE input
    selected_country = cve_country_dropdown.value  # Get selected country
    output_format = output_format_dropdown.value

    print(f"DEBUG: selected_country value at submission: {selected_country}")  # Add this debug
    print(f"DEBUG: output_format at submission: {output_format}")

    # Initialize output_file_path
    output_file_path = None

    # Custom output file name
    custom_file_name_value = custom_file_name.value.strip()

    # Determine if output should be saved to a file
    if output_to_file:
        if custom_file_name.value:
            output_file_path = f"output_files/{custom_file_name.value}.txt"
        else:
            output_file_path = f"output_files/default_output.txt"

    # Initialize variables for file content
    uploaded_file_content = None
    dropdown_file_content = None

    # Handling file upload (from the FileUpload widget)
    if file_input.value:
        print("DEBUG: file_input.value =", file_input.value)  # Add detailed debugging to see the structure
        try:
            # Extract the uploaded file content (since file_input.value is a tuple containing the file dictionary)
            uploaded_file = file_input.value[0]  # Get the first element of the tuple
            uploaded_file_content = uploaded_file['content'].tobytes().decode('utf-8')  # Convert memoryview to bytes and then decode to string
        except Exception as e:
            with status_output:
                clear_output()
                display(HTML(f"<b>Error extracting uploaded file content: {e}</b>"))
            return

    # Handling dropdown file selection (from the dropdown menu)
    if file_name_input.value and not file_input.value:
        # Only process dropdown selection if no file has been uploaded
        file_name_text = file_name_input.value
        file_path = os.path.join('input_files', file_name_text)

        # Ensure the file exists before attempting to read it
        if os.path.isfile(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    dropdown_file_content = f.read()
            except Exception as e:
                with status_output:
                    clear_output()
                    display(HTML(f"<b>Error reading file from dropdown: {e}</b>"))
                return
        else:
            with status_output:
                clear_output()
                display(HTML(f"<b>Error: File {file_name_text} not found.</b>"))
            return

    # Refang the raw input (defanged IP, URL, etc.)
    refanged_raw_text = sanitize_and_defang(raw_text, defang=False)

    # Ensure there is at least one valid input provided (CVE or IOC)
    if not refanged_raw_text and not cve_text and not uploaded_file_content and not dropdown_file_content:
        with status_output:
            clear_output()
            display(HTML("<b>Error: Please provide an IOC input or enter a CVE.</b>"))
        return

    # Check if the user is searching for a CVE and if a country is selected
    if cve_text:
        print(f"DEBUG: Inside CVE text handling, selected_country = {selected_country}")  # Added debug
        file_content = uploaded_file_content if uploaded_file_content else dropdown_file_content
        start_validation(
            output_to_file=output_to_file,
            custom_output_filename=custom_file_name_value,
            raw_input=refanged_raw_text,
            file_input=file_content,
            file_name_input=None,
            progress_bar=progress_bar,
            status_output=status_output,
            cve_text=cve_text,  # Pass the CVE to start_validation
            selected_country=selected_country,  # Pass the selected country
            output_format=output_format
        )
        return

    # Choose the appropriate file content if provided
    file_content = uploaded_file_content if uploaded_file_content else dropdown_file_content

    # Pass the output_file_path to start_validation
    start_validation(
        output_to_file=output_to_file,
        custom_output_filename=custom_file_name_value,
        raw_input=refanged_raw_text,
        file_input=file_content,  # Pass the processed file content (either from upload or dropdown)
        file_name_input=None,  # We don't need to pass the file name since content is already handled
        progress_bar=progress_bar,
        status_output=status_output,
        cve_text=cve_search_input.value,  # Get CVE input
        selected_country=cve_country_dropdown.value,
        output_format=output_format
    )

# Bind the button click events to the functions
submit_button.on_click(on_submit_button_clicked)
#partial_reset_button.on_click(partial_reset_inputs)
full_reset_button.on_click(full_reset_inputs)

# Create an HTML widget for the classification text with explicit centering
classification_text = widgets.HTML(
    value="<h3 style='text-align: center;'>UNCLASSIFIED//OUO</h3>",
    layout=widgets.Layout(margin='0px 0px 10px 0px', width='100%')
)


# Arrange the UI components in a box with a grey background
input_widgets = widgets.VBox([
    banner_widget,
    classification_text,
    raw_input,
    file_name_input,
    file_input,
    cve_country_dropdown,
    checkbox_vbox,
    submit_button,
    widgets.HBox([full_reset_button]),  # Both reset buttons in a horizontal box
    processing_label,
    status_output,
    image_widget,
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', background_color='#333333', border_radius='10px'))

app_layout = widgets.AppLayout(
    left_sidebar=input_widgets,
    center=None,
    right_sidebar=output,
    pane_widths=['300px', 0, 1],  # Reduced width of the left sidebar
    pane_heights=['80px', '1100px', '60px']  # Increased height to 900px
)

display(app_layout)